## Mapping tables to label amenity type
#### This notebook used for mapping.py


In [32]:
# %pip install pandas
import pandas as pd

## Load amenities dataset to get amenity types to be categorised

In [33]:
##load data
df = pd.read_csv('arcgis/amenities.csv')
df.shape
df.head()


/var/folders/dm/z_zvyzrd1sn7_mdfpl26_bvh0000gn/T/ipykernel_42371/3167001857.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('arcgis/amenities.csv')


,amenity_id,amenity_type,amenity_name,road_name,postal_code,geom_type,lon,lat,source_file,ADDRESSBUILDINGNAME,ADDRESSBLOCKHOUSENUMBER,LANDXADDRESSPOINT,LANDYADDRESSPOINT,PRECINCTPA,Shape__Area,Shape__Length,geometry_wkt,geometry_geojson
0,8d39567d-5ea6-5751-beec-88770a9be781,bus_depots,BISHAN DEPOT (SMRT),ANG MO KIO AVENUE 1,579991.0,Point,103.854302,1.355241,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.854302 1.355241),"{""type"":""Point"",""coordinates"":[103.85430205869..."
1,fbed27ca-8066-5e40-8cc5-d62cadfb738c,bus_depots,SBS TRANSIT BUS DEPOT (ANG MO KIO),ANG MO KIO STREET 63,569117.0,Point,103.847620,1.385921,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.84762 1.385921),"{""type"":""Point"",""coordinates"":[103.84761963749..."
2,fb09f377-194b-5c85-9942-cc74e9136575,bus_depots,ANG MO KIO DEPOT (SMRT),ANG MO KIO STREET 62,569140.0,Point,103.845621,1.387400,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.845621 1.3874),"{""type"":""Point"",""coordinates"":[103.84562108727..."
3,187dda5f-1316-5f06-854b-0718ea11f55c,bus_depots,SBS HOUGANG DEPOT,DEFU AVENUE 1,539536.0,Point,103.893088,1.346140,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.893088 1.34614),"{""type"":""Point"",""coordinates"":[103.89308806243..."
4,eb89bf2e-7583-5961-95b3-2c647513f432,bus_depots,SBS TRANSIT EAST DISTRICT BEDOK NORTH BUS PARK,BEDOK NORTH AVENUE 4,489946.0,Point,103.947977,1.338426,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.947977 1.338426),"{""type"":""Point"",""coordinates"":[103.94797719167..."


In [34]:
#check unique amenity types, these will be amenity subcategories
unique_amenity_types = df['amenity_type'].unique()
print(unique_amenity_types)
print(len(unique_amenity_types))
print(df['source_file'].count())
print(df.shape)


['bus_depots' 'bus_interchanges_terminals' 'bus_stops' 'childcare_clean'
 'chinese_temples' 'churches' 'community_clubs' 'concert_halls' 'courts'
 'fire_services' 'hdb_buildings' 'hdb_points_shp' 'higher_education'
 'historic_sites' 'hotels' 'indian_temples' 'kindergartens' 'libraries'
 'moe_schools' 'mosques' 'mrt_station_exits' 'other_institutions'
 'parkfacilities' 'police' 'post_offices' 'preschools' 'sikh_temples'
 'special_education' 'sports_centres' 'stadiums' 'swimming_complex'
 'synagogues' 'tourist_attractions']
33
37726
(37726, 18)


In [35]:
# map amenity_type as subcategories to categories
subcat_to_cat_mapping = {
    "emergency_services": ['fire_services'],
    "healthcare_facilities": [],
    "essential_services": [ 'childcare_clean', 'post_offices' , 'police' ],
    "residential": ['hdb_buildings'],
    "education_institutions": ['preschools', 'special_education' ,'moe_schools', 'higher_education', 'kindergartens' ],
    "transport_services": ['bus_depots','bus_interchanges_terminals','bus_stops', 'mrt_station_exits' ] ,
    "tourism": ['tourist_attractions', 'hotels', 'historic_sites' ],
    "community_spaces":['synagogues', 'sports_centres', 'stadiums' ,'swimming_complex' , 'churches', 'community_clubs', 'concert_halls', 'mosques', 'libraries', 'chinese_temples' , 'sikh_temples', 'indian_temples' , 'parkfacilities'],
    "government_services": ['courts'],
    "retail_services": ['hdb_points_shp'],
    "others": ['other_institutions']}

In [36]:
# decide on categories and (priority, weight)
# PRIORITY: lower number, more important --> based on importance to community 
## decided based on scale of 1 to 5, with 1 being critical to community and 5 being low impact on community if no access
# WEIGHT: higher number, more weight --> based on assumptinos made on usage/footfall 
## decided based on scale of 1 to 5, with 1 being low usage/footfall (used by avg person <1x a week) and 5 being high footfall (used by avg person 5-7x a week) with some manual override execptions (e.g. emergency services should always be available)
category_priority_weight = {
    "emergency_services": [1, 5],
    "healthcare_facilities": [1, 5],
    "essential_services": [1, 4],
    "residential": [1, 5],
    "education_institutions": [2, 4],
    "transport_services": [2, 4],
    "tourism": [5, 1],
    "community_spaces": [4, 2],
    "government_services": [3, 3],
    "financial_services": [3, 2],
    "retail_services": [4, 2],
    "recreation": [5, 2]
}

In [43]:
# given df with priority and weight, calculate importance score and add label
def amenity_importance(df, score_col="importance_score", score_label="importance_label", bins=[1, 2, 3, 5,8, 25], labels=["Negligible", "Low", "Moderate", "High", "Critical"]):
    # calculate importance score
    df[score_col] =  (df["amenity_weight"])**2/df["amenity_priority"]
    # add importance labels
    df[score_label] = pd.cut(df[score_col], bins=bins, labels=labels)

    return df

In [38]:
# create categoryDB
category_data = [(category, values[0], values[1]) for category, values in category_priority_weight.items()]
categoryDB = pd.DataFrame(category_data, columns=["amenity_category", "amenity_priority", "amenity_weight"])
print(categoryDB.head())

         amenity_category  amenity_priority  amenity_weight
0      emergency_services                 1               5
1   healthcare_facilities                 1               5
2      essential_services                 1               4
3             residential                 1               5
4  education_institutions                 2               4


In [39]:
# create subcat_to_catDB
subcat_to_catDB = pd.DataFrame([(subcat, cat) for cat, subcats in subcat_to_cat_mapping.items() for subcat in subcats], columns=["amenity_type", "amenity_category"])
print(subcat_to_catDB.head())


      amenity_type    amenity_category
0    fire_services  emergency_services
1  childcare_clean  essential_services
2     post_offices  essential_services
3           police  essential_services
4    hdb_buildings         residential


In [ ]:
# label ARCGIS data with category information
df = df.merge(subcat_to_catDB, on="amenity_type", how="left")
df.head()

,amenity_id,amenity_type,amenity_name,road_name,postal_code,geom_type,lon,lat,source_file,ADDRESSBUILDINGNAME,ADDRESSBLOCKHOUSENUMBER,LANDXADDRESSPOINT,LANDYADDRESSPOINT,PRECINCTPA,Shape__Area,Shape__Length,geometry_wkt,geometry_geojson,amenity_category
0,8d39567d-5ea6-5751-beec-88770a9be781,bus_depots,BISHAN DEPOT (SMRT),ANG MO KIO AVENUE 1,579991.0,Point,103.854302,1.355241,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.854302 1.355241),"{""type"":""Point"",""coordinates"":[103.85430205869...",transport_services
1,fbed27ca-8066-5e40-8cc5-d62cadfb738c,bus_depots,SBS TRANSIT BUS DEPOT (ANG MO KIO),ANG MO KIO STREET 63,569117.0,Point,103.847620,1.385921,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.84762 1.385921),"{""type"":""Point"",""coordinates"":[103.84761963749...",transport_services
2,fb09f377-194b-5c85-9942-cc74e9136575,bus_depots,ANG MO KIO DEPOT (SMRT),ANG MO KIO STREET 62,569140.0,Point,103.845621,1.387400,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.845621 1.3874),"{""type"":""Point"",""coordinates"":[103.84562108727...",transport_services
3,187dda5f-1316-5f06-854b-0718ea11f55c,bus_depots,SBS HOUGANG DEPOT,DEFU AVENUE 1,539536.0,Point,103.893088,1.346140,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.893088 1.34614),"{""type"":""Point"",""coordinates"":[103.89308806243...",transport_services
4,eb89bf2e-7583-5961-95b3-2c647513f432,bus_depots,SBS TRANSIT EAST DISTRICT BEDOK NORTH BUS PARK,BEDOK NORTH AVENUE 4,489946.0,Point,103.947977,1.338426,bus_depots.geojson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (103.947977 1.338426),"{""type"":""Point"",""coordinates"":[103.94797719167...",transport_services


In [ ]:
## add priority labels
df = df.merge(categoryDB, on="amenity_category", how="left")
df.head()

,amenity_id,amenity_type,amenity_name,road_name,postal_code,geom_type,lon,lat,source_file,ADDRESSBUILDINGNAME,...,LANDXADDRESSPOINT,LANDYADDRESSPOINT,PRECINCTPA,Shape__Area,Shape__Length,geometry_wkt,geometry_geojson,amenity_category,amenity_priority,amenity_weight
0,8d39567d-5ea6-5751-beec-88770a9be781,bus_depots,BISHAN DEPOT (SMRT),ANG MO KIO AVENUE 1,579991.0,Point,103.854302,1.355241,bus_depots.geojson,NaN,...,NaN,NaN,NaN,NaN,NaN,POINT (103.854302 1.355241),"{""type"":""Point"",""coordinates"":[103.85430205869...",transport_services,2.0,4.0
1,fbed27ca-8066-5e40-8cc5-d62cadfb738c,bus_depots,SBS TRANSIT BUS DEPOT (ANG MO KIO),ANG MO KIO STREET 63,569117.0,Point,103.847620,1.385921,bus_depots.geojson,NaN,...,NaN,NaN,NaN,NaN,NaN,POINT (103.84762 1.385921),"{""type"":""Point"",""coordinates"":[103.84761963749...",transport_services,2.0,4.0
2,fb09f377-194b-5c85-9942-cc74e9136575,bus_depots,ANG MO KIO DEPOT (SMRT),ANG MO KIO STREET 62,569140.0,Point,103.845621,1.387400,bus_depots.geojson,NaN,...,NaN,NaN,NaN,NaN,NaN,POINT (103.845621 1.3874),"{""type"":""Point"",""coordinates"":[103.84562108727...",transport_services,2.0,4.0
3,187dda5f-1316-5f06-854b-0718ea11f55c,bus_depots,SBS HOUGANG DEPOT,DEFU AVENUE 1,539536.0,Point,103.893088,1.346140,bus_depots.geojson,NaN,...,NaN,NaN,NaN,NaN,NaN,POINT (103.893088 1.34614),"{""type"":""Point"",""coordinates"":[103.89308806243...",transport_services,2.0,4.0
4,eb89bf2e-7583-5961-95b3-2c647513f432,bus_depots,SBS TRANSIT EAST DISTRICT BEDOK NORTH BUS PARK,BEDOK NORTH AVENUE 4,489946.0,Point,103.947977,1.338426,bus_depots.geojson,NaN,...,NaN,NaN,NaN,NaN,NaN,POINT (103.947977 1.338426),"{""type"":""Point"",""coordinates"":[103.94797719167...",transport_services,2.0,4.0


In [44]:
## calculate importance score and add label

df = amenity_importance(df)
df.head()

,amenity_id,amenity_type,amenity_name,road_name,postal_code,geom_type,lon,lat,source_file,ADDRESSBUILDINGNAME,...,PRECINCTPA,Shape__Area,Shape__Length,geometry_wkt,geometry_geojson,amenity_category,amenity_priority,amenity_weight,importance_score,importance_label
0,8d39567d-5ea6-5751-beec-88770a9be781,bus_depots,BISHAN DEPOT (SMRT),ANG MO KIO AVENUE 1,579991.0,Point,103.854302,1.355241,bus_depots.geojson,NaN,...,NaN,NaN,NaN,POINT (103.854302 1.355241),"{""type"":""Point"",""coordinates"":[103.85430205869...",transport_services,2.0,4.0,8.0,High
1,fbed27ca-8066-5e40-8cc5-d62cadfb738c,bus_depots,SBS TRANSIT BUS DEPOT (ANG MO KIO),ANG MO KIO STREET 63,569117.0,Point,103.847620,1.385921,bus_depots.geojson,NaN,...,NaN,NaN,NaN,POINT (103.84762 1.385921),"{""type"":""Point"",""coordinates"":[103.84761963749...",transport_services,2.0,4.0,8.0,High
2,fb09f377-194b-5c85-9942-cc74e9136575,bus_depots,ANG MO KIO DEPOT (SMRT),ANG MO KIO STREET 62,569140.0,Point,103.845621,1.387400,bus_depots.geojson,NaN,...,NaN,NaN,NaN,POINT (103.845621 1.3874),"{""type"":""Point"",""coordinates"":[103.84562108727...",transport_services,2.0,4.0,8.0,High
3,187dda5f-1316-5f06-854b-0718ea11f55c,bus_depots,SBS HOUGANG DEPOT,DEFU AVENUE 1,539536.0,Point,103.893088,1.346140,bus_depots.geojson,NaN,...,NaN,NaN,NaN,POINT (103.893088 1.34614),"{""type"":""Point"",""coordinates"":[103.89308806243...",transport_services,2.0,4.0,8.0,High
4,eb89bf2e-7583-5961-95b3-2c647513f432,bus_depots,SBS TRANSIT EAST DISTRICT BEDOK NORTH BUS PARK,BEDOK NORTH AVENUE 4,489946.0,Point,103.947977,1.338426,bus_depots.geojson,NaN,...,NaN,NaN,NaN,POINT (103.947977 1.338426),"{""type"":""Point"",""coordinates"":[103.94797719167...",transport_services,2.0,4.0,8.0,High
